# IS IT POSSIBLE TO FORECAST OR IT'S JUST A DREAM

<img src="https://www.webtekno.com/images/editor/default/0002/48/a5c0386ffcd7114174d86c924aff21ae90cba989.jpeg" width=600 height=200>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import warnings
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import datetime
import shap  # package used to calculate Shap values
from statsmodels.stats.outliers_influence import variance_inflation_factor 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_g=pd.read_csv("/kaggle/input/stock-market-google-facebook-twitter/google_data.csv")
df_f=pd.read_csv("/kaggle/input/stock-market-google-facebook-twitter/facebook_data.csv")
df_t=pd.read_csv("/kaggle/input/stock-market-google-facebook-twitter/twtr_data.csv")
df_g["date"]=pd.to_datetime(df_g["date"])
df_f["date"]=pd.to_datetime(df_f["date"])
df_t["date"]=pd.to_datetime(df_t["date"])

### Stock market prediction is the act of trying to determine the future value of a company stock or other financial instrument traded on an exchange. The successful prediction of a stock's future price could yield significant profit. The efficient-market hypothesis suggests that stock prices reflect all currently available information and any price changes that are not based on newly revealed information thus are inherently unpredictable. Others disagree and those with this viewpoint possess myriad methods and technologies which purportedly allow them to gain future price information.

### Prediction Methods
> Prediction methodologies fall into three broad categories which can (and often do) overlap. They are fundamental analysis, technical analysis (charting) and technological methods.



# On this Notebook we use Technical analysis and Machine learning methods

### **Technical analysis**
<br>
Technical analysts or chartists are not concerned with any of the company's fundamentals. They seek to determine the future price of a stock based solely on the trends of the past price (a form of time series analysis). Numerous patterns are employed such as the head and shoulders or cup and saucer. Alongside the patterns, techniques are used such as the exponential moving average (EMA), oscillators, support and resistance levels or momentum and volume indicators. Candle stick patterns, believed to have been first developed by Japanese rice merchants, are nowadays widely used by technical analysts. Technical analysis is rather used for short-term strategies, than the long-term ones. And therefore, it is far more prevalent in commodities and forex markets where traders focus on short-term price movements. There are some basic assumptions used in this analysis, first being that everything significant about a company is already priced into the stock, other being that the price moves in trends and lastly that history (of prices) tends to repeat itself which is mainly because of the market psychology.


### First we need to add most popular indicator to our Dataset


### Create functions that we need

In [ ]:
dataset=df_g  # which dataset we use on model

In [ ]:
def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True
    return X.iloc[:, variables]



def RSI(df, n):
    close = df['close']
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi


def stochastic(df, k, d):
    df = df.copy()
    low_min  = df['low'].rolling(window=k).min()
    high_max = df['high'].rolling( window=k).max()
    df['stoch_k'] = 100 * (df['close'] - low_min)/(high_max - low_min)
    df['stoch_d'] = df['stoch_k'].rolling(window=d).mean()
    return df

In [ ]:
dataset.loc[(dataset["close"]>dataset["close"].shift(-1)), "target"] =  1
dataset.loc[(dataset["close"]<=dataset["close"].shift(-1)), "target"] =  0
dataset["target"]=dataset["target"].fillna(0)
dataset["target"]=dataset["target"].astype(int)


dataset.drop(["exchange","symbol","adj_volume","adj_open","adj_close","adj_low","adj_high"],axis=1,inplace=True)

dataset=dataset.sort_values(by=["date"],ascending=True)
dataset['SMA_5'] = dataset['close'].rolling(5).mean()
dataset['SMA_7'] = dataset['close'].rolling(7).mean()
dataset['SMA_10'] = dataset['close'].rolling(10).mean()
dataset['SMA_12'] = dataset['close'].rolling(12).mean()
dataset['SMA_20'] = dataset['close'].rolling(20).mean()
dataset['SMA_21'] = dataset['close'].rolling(21).mean()
dataset['SMA_23'] = dataset['close'].rolling(23).mean()
dataset['SMA_26'] = dataset['close'].rolling(26).mean()
dataset['SMA_50'] = dataset['close'].rolling(50).mean()
dataset['SMA_200'] = dataset['close'].rolling(200).mean()
dataset['SMA_233'] = dataset['close'].rolling(233).mean()
dataset["rstd"] = dataset['close'].rolling(20).std()
dataset['BB_LOWER'] = dataset['SMA_20']-(2*dataset["rstd"])
dataset['BB_UPPER'] = dataset['SMA_20']+(2*dataset["rstd"])
dataset['RSI_14'] = RSI(dataset,14)
dataset['RSI_9'] = RSI(dataset,9)
dataset['RSI_20'] = RSI(dataset,20)
dataset["MACD"] = dataset['SMA_12']-dataset['SMA_26']
dataset["MACD_signal"] = dataset['MACD'].rolling(9).mean()
dataset=stochastic(dataset, k=14, d=3)

dataset["Typical_Price"]=(dataset["high"]+dataset["low"]+dataset["close"])/3
dataset["Row_Money_Flow"]=dataset["Typical_Price"]*dataset["volume"]
dataset["Money_Flow_Ratio"]=dataset["Row_Money_Flow"].rolling(14).max()/dataset["Row_Money_Flow"].rolling(14).min()
dataset["MFI_num"]=100 - 100 / (1 + dataset["Money_Flow_Ratio"])
dataset.loc[(dataset["MFI_num"]<=30), "MFI_AL"] =  1
dataset.loc[(dataset["MFI_num"]>30), "MFI_AL"] =  0

dataset=dataset.sort_values(by=["date"],ascending=False)

dataset["close_diff"]=dataset["close"]-dataset["close"].shift(-1)
dataset["close_perc"]=(dataset["close"]-dataset["close"].shift(-1))/dataset["close"].shift(-1)
dataset["close_open_diff"]=dataset["close"]-dataset["open"]
dataset["close_open_perc"]=(dataset["close"]-dataset["open"])/dataset["open"]
dataset.loc[(dataset["close"]>dataset["close"].shift(-1)), "MOM1"] =  1
dataset.loc[(dataset["close"]<=dataset["close"].shift(-1)), "MOM1"] =  0
dataset.loc[(dataset["close"]>dataset["close"].shift(-2)), "MOM2"] =  1
dataset.loc[(dataset["close"]<=dataset["close"].shift(-2)), "MOM2"] =  0
dataset.loc[(dataset["close"]>dataset["close"].shift(-3)), "MOM3"] =  1
dataset.loc[(dataset["close"]<=dataset["close"].shift(-3)), "MOM3"] =  0
dataset.loc[(dataset["close"]>dataset["close"].shift(-4)), "MOM4"] =  1
dataset.loc[(dataset["close"]<=dataset["close"].shift(-4)), "MOM4"] =  0
dataset.loc[(dataset["close"]>dataset["close"].shift(-5)), "MOM5"] =  1
dataset.loc[(dataset["close"]<=dataset["close"].shift(-5)), "MOM5"] =  0
dataset["MOM_TOP5"]=dataset["MOM1"]+dataset["MOM2"]+dataset["MOM3"]+dataset["MOM4"]+dataset["MOM5"]
dataset.loc[(dataset["SMA_5"]>dataset["SMA_10"]), "SMA_5_10_COMP"] =  1
dataset.loc[(dataset["SMA_5"]<=dataset["SMA_10"]), "SMA_5_10_COMP"] =  0
dataset.loc[(dataset["SMA_10"]>dataset["SMA_20"]), "SMA_10_20_COMP"] =  1
dataset.loc[(dataset["SMA_10"]<=dataset["SMA_20"]), "SMA_10_20_COMP"] =  0
dataset.loc[(dataset["SMA_20"]>dataset["SMA_26"]), "SMA_20_26_COMP"] =  1
dataset.loc[(dataset["SMA_20"]<=dataset["SMA_26"]), "SMA_20_26_COMP"] =  0
dataset.loc[(dataset["SMA_20"]>dataset["SMA_50"]), "SMA_20_50_COMP"] =  1
dataset.loc[(dataset["SMA_20"]<=dataset["SMA_50"]), "SMA_20_50_COMP"] =  0
dataset.loc[(dataset["SMA_50"]>dataset["SMA_200"]), "SMA_50_200_COMP"] =  1
dataset.loc[(dataset["SMA_50"]<=dataset["SMA_200"]), "SMA_50_200_COMP"] =  0
dataset.loc[(dataset["RSI_9"]<=30), "RSI_9_AL"] =  1
dataset.loc[(dataset["RSI_9"]>30), "RSI_9_AL"] =  0
dataset.loc[(dataset["RSI_14"]<=30), "RSI_14_AL"] =  1
dataset.loc[(dataset["RSI_14"]>30), "RSI_14_AL"] =  0
dataset.loc[(dataset["RSI_20"]<=30), "RSI_20_AL"] =  1
dataset.loc[(dataset["RSI_20"]>30), "RSI_20_AL"] =  0
dataset.loc[(dataset["MACD_signal"]<=dataset["MACD"]), "MACD_signal_comp_al"] =  1
dataset.loc[(dataset["MACD_signal"]>dataset["MACD"]), "MACD_signal_comp_al"] =  0


dataset.loc[(dataset["close"]<=dataset["SMA_5"]), "CLOS_SMA5_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_5"]), "CLOS_SMA5_comp_al"] =  1
dataset.loc[(dataset["close"]<=dataset["SMA_20"]), "CLOS_SMA20_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_20"]), "CLOS_SMA20_comp_al"] =  1
dataset.loc[(dataset["close"]<=dataset["SMA_21"]), "CLOS_SMA21_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_21"]), "CLOS_SMA21_comp_al"] =  1
dataset.loc[(dataset["close"]<=dataset["SMA_23"]), "CLOS_SMA23_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_23"]), "CLOS_SMA23_comp_al"] =  1
dataset.loc[(dataset["close"]<=dataset["SMA_26"]), "CLOS_SMA26_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_26"]), "CLOS_SMA26_comp_al"] =  1
dataset.loc[(dataset["close"]<=dataset["SMA_50"]), "CLOS_SMA50_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_50"]), "CLOS_SMA50_comp_al"] =  1
dataset.loc[(dataset["close"]<=dataset["SMA_200"]), "CLOS_SMA200_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_200"]), "CLOS_SMA200_comp_al"] =  1
dataset.loc[(dataset["close"]<=dataset["SMA_233"]), "CLOS_SMA200_comp_al"] =  0
dataset.loc[(dataset["close"]>dataset["SMA_233"]), "CLOS_SMA233_comp_al"] =  1

dataset.loc[(dataset["BB_LOWER"]>=dataset["close"]), "BB_al"] =  1
dataset.loc[(dataset["BB_LOWER"]<dataset["close"]), "BB_al"] =  0 
dataset["open_close_ratio"] = dataset['open']/dataset['close']
dataset["high_low_diff"]=dataset["high"]-dataset["low"]
dataset["high_low_ratio"]=(dataset["high"]-dataset["low"])/dataset["low"]
dataset["high_low_ratio1"]=(dataset["high"].shift(-1)-dataset["low"].shift(-1))/dataset["low"].shift(-1)
dataset["high_high1_ratio"]=(dataset["high"]-dataset["high"].shift(-1))/dataset["high"].shift(-1)
dataset["high_high2_ratio"]=(dataset["high"]-dataset["high"].shift(-2))/dataset["high"].shift(-2)
dataset["high_high3_ratio"]=(dataset["high"]-dataset["high"].shift(-3))/dataset["high"].shift(-3)
dataset["high_close_ratio"]=(dataset["high"]-dataset["close"])/dataset["close"]
dataset.loc[(dataset["RSI_14"]<=25) & (dataset['SMA_23']>dataset['SMA_26']), "MACD_RSI_SIGNAL"] =  1
dataset.loc[(dataset["RSI_14"]>25) | (dataset['SMA_23']<=dataset['SMA_26']), "MACD_RSI_SIGNAL"] =  0
dataset=dataset.dropna()
dataset.sort_values(by="date",ascending=True)
columns=dataset.columns.drop("target")
dataset[columns]=dataset[columns].shift(-1)
dataset=dataset.dropna()
dataset=dataset.drop(["date"],axis=1)


In [ ]:

r_s=42  
dataset=dataset.dropna()
dataset.columns[dataset.isnull().any()]

train_x=dataset.drop("target",axis=1)
#correlation(train_x, 0.95)
train_y=dataset["target"]
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.20, random_state=r_s)


xgb=GradientBoostingClassifier(random_state=42)
xgb=xgb.fit(X_train,y_train)

# Create object that can calculate shap values
explainer = shap.TreeExplainer(xgb)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_val)

shap_sum = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame([X_val.columns.tolist(), shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)
important_feature=importance_df["column_name"].head(1)

#-----------------------------------------------------------------------------------------------------------   

my_regressors=[ 
               GradientBoostingClassifier(random_state=r_s),
               GradientBoostingClassifier(learning_rate=0.05, n_estimators=120,max_depth=9, min_samples_split=1200,
                                            min_samples_leaf=60, subsample=0.85, max_features=7,random_state=r_s),
               GradientBoostingClassifier(learning_rate=0.01, n_estimators=600,max_depth=9, min_samples_split=1200,
                                            min_samples_leaf=60, subsample=0.85,max_features=7,random_state=r_s),
               GradientBoostingClassifier(learning_rate=0.005, n_estimators=1200,max_depth=9, min_samples_split=1200,
                                            min_samples_leaf=60, subsample=0.85, max_features=7,warm_start=True,random_state=r_s),
               GradientBoostingClassifier(learning_rate=0.005, n_estimators=1500,max_depth=9, min_samples_split=1200, 
                                            min_samples_leaf=60, subsample=0.85, max_features=7,warm_start=True,random_state=r_s),
               GradientBoostingClassifier(learning_rate=100, n_estimators=500,max_depth=5,random_state=r_s),
               GradientBoostingClassifier(learning_rate=0.01, n_estimators=250,max_depth=7,random_state=r_s),
               GradientBoostingClassifier(learning_rate=0.1, n_estimators=5,max_depth=3,random_state=r_s),
               RandomForestClassifier(random_state=r_s),
               RandomForestClassifier(n_estimators=200,random_state=r_s),
               RandomForestClassifier(n_estimators=200,min_samples_leaf=3,max_features=0.5,random_state=r_s),
               RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                                        min_impurity_decrease=0.0, min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
                                        oob_score=False, random_state=r_s, verbose=0,
                                        warm_start=False),
               RandomForestClassifier(n_estimators=50,max_features="sqrt",random_state=r_s),
               RandomForestClassifier(bootstrap=True,max_depth=50,n_estimators=25,max_features=5,random_state=r_s),
               AdaBoostClassifier(random_state=r_s),
               ExtraTreesClassifier(random_state=r_s),
               LGBMClassifier(random_state=r_s), 
               LGBMClassifier(min_child_weight=1e-05,reg_alpha=1,reg_lambda=100,scale_pos_weight=2,random_state=r_s),       
               CatBoostClassifier(eval_metric='Accuracy',logging_level='Silent',random_state=r_s), 
               CatBoostClassifier(border_count=32,depth=3,iterations=250,l2_leaf_reg=3,learning_rate=0.03,eval_metric='Accuracy',logging_level='Silent',random_state=r_s),         
               XGBClassifier(random_state=r_s),
               XGBClassifier(scale_pos_weight=1,learning_rate=0.01,colsample_bytree = 0.4,subsample = 0.8,
                                objective='binary:logistic',n_estimators=1000,reg_alpha = 0.3,max_depth=4,gamma=10,random_state=r_s),
               XGBClassifier(learning_rate =0.01,n_estimators=5000,max_depth=4,min_child_weight=6,gamma=0,subsample=0.8,
                                colsample_bytree=0.8,reg_alpha=0.005,objective= 'binary:logistic',nthread=4,
                                scale_pos_weight=1,seed=27,random_state=r_s),
              XGBClassifier(max_depth=3,min_child_weight=10,gamma=5,subsample=0.6,
                                colsample_bytree=0.8,random_state=r_s),
             XGBClassifier(learning_rate =10,gamma=1.5,subsample=1.0,
                                colsample_bytree=0.6,random_state=r_s),
               #MLPClassifier(random_state=r_s),
               KNeighborsClassifier(3),
               SVC(kernel="linear", C=0.025,random_state=r_s),
               SVC(gamma=2, C=1,random_state=r_s),
               GaussianProcessClassifier(1.0 * RBF(1.0),random_state=r_s),
               DecisionTreeClassifier(random_state=r_s),
               DecisionTreeClassifier(criterion="entropy",max_depth=4,random_state=r_s),
               GaussianNB(),
               QuadraticDiscriminantAnalysis(),
               LogisticRegression(random_state=r_s)
              ]

regressors=[]

for my_regressor in my_regressors:
    regressors.append(my_regressor)


scores_val=[]
scores_train=[]
MAE=[]
MSE=[]
RMSE=[]



for model in regressors:
    pipe=Pipeline([("scaler",RobustScaler()),("regressor",model)])
    scores_val.append(pipe.fit(X_train,y_train).score(X_val,y_val))
    scores_train.append(pipe.fit(X_train,y_train).score(X_train,y_train))
    y_pred=pipe.predict(X_val)
    scores_val.append(pipe.fit(X_train,y_train).score(X_val,y_val))
    scores_train.append(pipe.fit(X_train,y_train).score(X_train,y_train))
    y_pred=pipe.predict(X_val)
    MAE.append(mean_absolute_error(y_val,y_pred))
    MSE.append(mean_squared_error(y_val,y_pred))
    RMSE.append(np.sqrt(mean_squared_error(y_val,y_pred)))



results=zip(scores_val,scores_train,MAE,MSE,RMSE)
results=list(results)
results_score_val=[item[0] for item in results]
results_score_train=[item[1] for item in results]
results_MAE=[item[2] for item in results]
results_MSE=[item[3] for item in results]
results_RMSE=[item[4] for item in results]


df_results=pd.DataFrame({"Algorithm":regressors,"Training Score":results_score_train,"Validation Score":results_score_val,"MAE":results_MAE,"MSE":results_MSE,"RMSE":results_RMSE})
best_models=df_results.sort_values(by="Validation Score",ascending=False)
best_model=best_models.iloc[0]
#print(best_model)
best_model_name=best_models.iloc[0][0]
pipe_best=Pipeline([("scaler",RobustScaler()),("regressor",best_model_name)])
best_model_learn=pipe_best.fit(X_train,y_train)
y_pred_best=best_model_learn.predict(X_val)


from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report")
print(classification_report(y_val, y_pred_best)) 


In [ ]:
cf_matrix=confusion_matrix(y_val, y_pred_best)
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
plt.show()